# Agents - Make OpenAI Do Things For you

*[Source](https://langchain.readthedocs.io/en/latest/modules/agents/getting_started.html)*
* **Agent** - Agents use an LLM to determine which actions to take and in what order. An action can either be using a tool and observing its output, or returning to the user.

Parameters when creating an agent:
* **Tool:** A function that performs a specific duty. This can be things like: Google Search, Database lookup, Python REPL, other chains. The interface for a tool is currently a function that is expected to have a string as an input, with a string as an output.
* **LLM:** The language model powering the agent.
* **Agent:** The agent to use. This should be a string that references a support agent class. Because this notebook focuses on the simplest, highest level API, this only covers using the standard supported agents. If you want to implement a custom agent, see the documentation for custom agents (coming soon).

In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_groq import ChatGroq
chat_google = ChatGoogleGenerativeAI(temperature=0.9, model='gemini-pro')
chat_groq = ChatGroq(temperature=0, model='llama3-groq-8b-8192-tool-use-preview')


d:\code\python\SETI\GenAI\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 2. SERP API

In [2]:
from langchain.agents import load_tools
from langchain_community.utilities import GoogleSerperAPIWrapper
from dotenv import load_dotenv
import os
# Load the environment variables from .env
load_dotenv()

"SERPER_API_KEY" in os.environ

True

In [3]:
search = GoogleSerperAPIWrapper(k=3)
results = search.results("medallas obtenidas en los juegos olimpicos 2024",)
results

{'searchParameters': {'q': 'medallas obtenidas en los juegos olimpicos 2024',
  'gl': 'us',
  'hl': 'en',
  'type': 'search',
  'num': 3,
  'engine': 'google'},
 'organic': [{'title': 'Conteo de medallas - Medallero Olímpico de París 2024',
   'link': 'https://olympics.com/es/paris-2024/medallas',
   'snippet': 'Medallero oficial de los Juegos Olímpicos París 2024 (26 de julio-11 de agosto de 2024). Descubre qué atletas se llevan medallas a casa y ...',
   'date': '2 days ago',
   'attributes': {'Missing': 'obtenidas | Show results with:obtenidas'},
   'sitelinks': [{'title': 'Medallistas',
     'link': 'https://olympics.com/es/paris-2024/medallas/medallistas'},
    {'title': 'Gimnasia artística',
     'link': 'https://olympics.com/es/paris-2024/medallas/gimnasia-artistica'},
    {'title': 'Atletismo',
     'link': 'https://olympics.com/es/paris-2024/medallas/atletismo'},
    {'title': 'Boxeo',
     'link': 'https://olympics.com/es/paris-2024/medallas/boxeo'}],
   'position': 1},
  {'t

In [4]:

tools = load_tools(tool_names=["llm-math","google-serper"],llm=chat_groq)
print(tools)

[Tool(name='Calculator', description='Useful for when you need to answer questions about math.', func=<bound method Chain.run of LLMMathChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['question'], template='Translate a math problem into a expression that can be executed using Python\'s numexpr library. Use the output of running this code to answer the question.\n\nQuestion: ${{Question with math problem.}}\n```text\n${{single line mathematical expression that solves the problem}}\n```\n...numexpr.evaluate(text)...\n```output\n${{Output of running the code}}\n```\nAnswer: ${{Answer}}\n\nBegin.\n\nQuestion: What is 37593 * 67?\n```text\n37593 * 67\n```\n...numexpr.evaluate("37593 * 67")...\n```output\n2518731\n```\nAnswer: 2518731\n\nQuestion: 37593^(1/5)\n```text\n37593**(1/5)\n```\n...numexpr.evaluate("37593**(1/5)")...\n```output\n8.222831614237718\n```\nAnswer: 8.222831614237718\n\nQuestion: {question}\n'), llm=ChatGroq(client=<groq.resources.chat.completions.Completi

In [5]:
llm_with_tools = chat_groq.bind_tools(tools)
llm_with_tools.invoke("calcular el 2*3")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_1x30', 'function': {'arguments': '{"config": {"args": [2, 3]}}', 'name': 'Calculator'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 477, 'total_tokens': 510, 'completion_time': 0.030056098, 'prompt_time': 0.06481874, 'queue_time': None, 'total_time': 0.094874838}, 'model_name': 'llama3-groq-8b-8192-tool-use-preview', 'system_fingerprint': 'fp_260dc69250', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-63c20835-2352-4d03-b7ef-242122109d8d-0', tool_calls=[{'name': 'Calculator', 'args': {'config': {'args': [2, 3]}}, 'id': 'call_1x30', 'type': 'tool_call'}], usage_metadata={'input_tokens': 477, 'output_tokens': 33, 'total_tokens': 510})

In [16]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import initialize_agent


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)


agent = create_tool_calling_agent(chat_groq, tools, prompt,)
agent_executor = AgentExecutor(agent=agent, tools=tools)

In [17]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad', optional=True)]

In [18]:
agent_executor.invoke({"input":"search about olympic games 2024"})

TypeError: Object of type CallbackManagerForToolRun is not JSON serializable

In [10]:
from langchain_core.messages import SystemMessage
from langgraph.prebuilt import create_react_agent

system_message = "You are a helpful assistant. Respond only in Spanish."
# This could also be a SystemMessage object
# system_message = SystemMessage(content="You are a helpful assistant. Respond only in Spanish.")

app = create_react_agent(chat_groq, tools, state_modifier=system_message)

query = "list the best pasta restaurants"


messages = app.invoke({"messages": [("human", query)]})


In [11]:
for message in messages["messages"]:
    print (message)

content='list the best pasta restaurants' id='264f8cb6-d84f-42ac-9a6d-c46d2481cd9f'
content="I'm sorry but I do not have the capability to perform this task for you, I am happy to help you with any other queries you may have." response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 487, 'total_tokens': 519, 'completion_time': 0.029052988, 'prompt_time': 0.066332591, 'queue_time': None, 'total_time': 0.095385579}, 'model_name': 'llama3-groq-8b-8192-tool-use-preview', 'system_fingerprint': 'fp_260dc69250', 'finish_reason': 'stop', 'logprobs': None} id='run-7d65716a-8e2a-4ce6-bd56-3759a93fb4be-0' usage_metadata={'input_tokens': 487, 'output_tokens': 32, 'total_tokens': 519}


In [12]:
agent_executor = AgentExecutor(agent=agent, tools=tools)

In [13]:
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.prebuilt import create_react_agent


